In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import uproot
import awkward as ak
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.n_train = 0
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, output_dim)
        )

    def forward(self, x):
        return self.fc(x)


In [2]:
def select_CConly(arr):
    return np.where(arr == 0)

def select_NConly(arr):
    return np.where(arr == 1)

In [7]:
required_truth_branches = ['ccnc_truth', 'nuPDG_truth',
                        'enu_truth', 
                        'nu_dcosx_truth', 'nu_dcosy_truth', 'nu_dcosz_truth']

required_reco_branches = [
                        'shwr_length_pandoraShower', 
                        'shwr_startdcosx_pandoraShower', 'shwr_startdcosy_pandoraShower', 'shwr_startdcosz_pandoraShower', 
                        'shwr_startx_pandoraShower', 'shwr_starty_pandoraShower', 'shwr_startz_pandoraShower', 
                        'shwr_totEng_pandoraShower',
                        'trkendx_pandoraTrack', 'trkendy_pandoraTrack', 'trkendz_pandoraTrack', 
                        'trkstartdcosx_pandoraTrack', 'trkstartdcosy_pandoraTrack', 'trkstartdcosz_pandoraTrack', 
                        'trkstartx_pandoraTrack', 'trkstarty_pandoraTrack', 'trkstartz_pandoraTrack']

In [8]:
settings = {
    "event_type": "CC", # "CC", "NC", or "ALL"
    "epochs": 50,
    "batch_size": 100,
    "loss_function": nn.MSELoss() # nn.MSELoss() or CustomMSELoss() or CustomL1Loss()
}

In [9]:
# Open root file, specifically the anatree root tree
with uproot.open("../../../data/atmsim/anatree_hd_AV_2dot5_random_sum_300k_new.root:analysistree/anatree") as tree:
    # Create empty dictionaries to store branch information in
    df_truth = {}
    df_reco = {}
    # Store all MC related branch-info in the truth dictionary
    for branch_name in required_truth_branches: 
        df_truth[branch_name] = tree[branch_name].array()
    # Store all reco related branch-info in the reco dictionary
    for branch_name in required_reco_branches:
        df_reco[branch_name] = tree[branch_name].array()


    for key in required_truth_branches:
        df_truth[key] = ak.to_numpy(ak.sum(df_truth[key] * (df_truth[key] != -999), axis=1))
    
    for key in required_reco_branches:
        if key == 'shwr_totEng_pandoraShower' or key == 'shwr_dedx_pandoraShower':
            df_reco[key] = ak.to_numpy(ak.sum(ak.sum(df_reco[key] * (df_reco[key] != -999), axis=1), axis=1))
        else:
            df_reco[key] = ak.to_numpy(ak.sum(df_reco[key] * (df_reco[key] != -999), axis=1))
    
    if settings["event_type"] == "CC":
        ana_indices = select_CConly(df_truth['ccnc_truth'])
        #ana_indices = np.where(np.logical_and(df_truth['ccnc_truth'] == 0, df_truth['nuPDG_truth'] == 14))[0]
    elif settings["event_type"] == "NC":
        ana_indices = select_NConly(df_truth['ccnc_truth'])
    elif settings["event_type"] == "ALL":
        ana_indices = np.arange(0,len(df_truth['ccnc_truth']))
    else:
        raise NameError
    required_truth_branches_ml = ['enu_truth', 'nu_dcosx_truth', 'nu_dcosy_truth', 'nu_dcosz_truth']
    df_truth_values = [df_truth[key][ana_indices] for key in required_truth_branches_ml]
    df_reco_values = [df_reco[key][ana_indices] for key in required_reco_branches]
    
    Y = np.stack(df_truth_values, axis=0).T
    X = np.stack(df_reco_values, axis=0).T
    
    print(X.shape, Y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    scaler = StandardScaler() 

    # Fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train) 
    X_test = scaler.transform(X_test)

    try:
        import os
        os.mkdir(f"{settings}")
    except FileExistsError:
        pass
    torch.save(X_test, f"{settings}/X_test.pt")
    torch.save(y_test, f"{settings}/y_test.pt")

(166133, 17) (166133, 4)


In [10]:
model = RegressionModel(X.shape[1], Y.shape[1])
model.n_train = len(y_train)

criterion = settings['loss_function']
optimizer = optim.Adam(model.parameters(), lr=5e-4)

num_epochs = settings['epochs']
batch_size = settings['batch_size']

best_loss = None
nonimprovement_counter = 0
for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = torch.tensor(X_train[i:i+batch_size], dtype=torch.float32)
        targets = torch.tensor(y_train[i:i+batch_size], dtype=torch.float32)
        #print(targets.shape)
        
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    if epoch == 0:
        best_loss = loss
    elif loss < best_loss:
        best_loss = loss       
        nonimprovement_counter = 0
    else:
        nonimprovement_counter += 1
        pass
        
    if nonimprovement_counter >= 20:
        print(f"No improvement for 10 epochs")
        break
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, nonimpovement_counter: {nonimprovement_counter}')

Epoch [1/50], Loss: 0.3929, nonimpovement_counter: 0
Epoch [2/50], Loss: 0.3031, nonimpovement_counter: 0
Epoch [3/50], Loss: 0.3322, nonimpovement_counter: 1
Epoch [4/50], Loss: 0.3836, nonimpovement_counter: 2
Epoch [5/50], Loss: 0.4625, nonimpovement_counter: 3
Epoch [6/50], Loss: 0.4957, nonimpovement_counter: 4
Epoch [7/50], Loss: 0.5101, nonimpovement_counter: 5
Epoch [8/50], Loss: 0.5270, nonimpovement_counter: 6
Epoch [9/50], Loss: 0.5362, nonimpovement_counter: 7
Epoch [10/50], Loss: 0.5325, nonimpovement_counter: 8
Epoch [11/50], Loss: 0.5182, nonimpovement_counter: 9
Epoch [12/50], Loss: 0.5124, nonimpovement_counter: 10
Epoch [13/50], Loss: 0.4997, nonimpovement_counter: 11
Epoch [14/50], Loss: 0.4804, nonimpovement_counter: 12
Epoch [15/50], Loss: 0.4788, nonimpovement_counter: 13
Epoch [16/50], Loss: 0.4489, nonimpovement_counter: 14
Epoch [17/50], Loss: 0.4324, nonimpovement_counter: 15
Epoch [18/50], Loss: 0.4176, nonimpovement_counter: 16
Epoch [19/50], Loss: 0.4045, n

In [11]:
# Save model parameters
torch.save(model, f"{settings}/model.pt")